In [ ]:
%pip install langchain-core langchain-openai langchain-exa

In [2]:
import getpass
import os

# Get API keys securely using getpass
openai_api_key = getpass.getpass(prompt="Enter your OPENAI_API_KEY: ")
exa_api_key = getpass.getpass(prompt="Enter your EXA_API_KEY: ")
langsmith_api_key = getpass.getpass(prompt ="Enter your langsmith api Key" )
# Set environment variables
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["EXA_API_KEY"] = exa_api_key
os.environ["LANGCHAIN_API_KEY"] = langsmith_api_key

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "test-default"

In [4]:
from langchain_exa import ExaSearchRetriever

retriever = ExaSearchRetriever(k=3, highlights=True)
documents = retriever.invoke("what is latest breakthrough in AI")

In [5]:
print(documents[0].metadata["highlights"])

['Big data is speeding up the AI development process, and we may be seeing more integration of AI technology in our everyday lives relatively soon. While much of this technology is still fairly rudimentary at the moment, we can expect sophisticated AI to one day significantly impact our everyday lives. Here are 6 ways AI might affect us in the future.    ']


In [6]:
print(documents[0].metadata["url"])

http://ai2080.blogspot.com/2018/02/next-generation-technology.html


In [7]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda


retriever = ExaSearchRetriever(k=3, highlights=True)

document_prompt = PromptTemplate.from_template("""
<source>
    <url>{url}</url>
    <highlights>{highlights}</highlights>
</source>
""")

document_chain = RunnableLambda(
    lambda document: {
        "highlights": document.metadata["highlights"], 
        "url": document.metadata["url"]
    }
) | document_prompt

retrieval_chain = retriever | document_chain.map() | (lambda docs: "\n".join([i.text for i in docs]))

print(retrieval_chain.invoke("what is latest breakthrough in AI"))


<source>
    <url>http://ai2080.blogspot.com/2018/02/next-generation-technology.html</url>
    <highlights>['Big data is speeding up the AI development process, and we may be seeing more integration of AI technology in our everyday lives relatively soon. While much of this technology is still fairly rudimentary at the moment, we can expect sophisticated AI to one day significantly impact our everyday lives. Here are 6 ways AI might affect us in the future.    ']</highlights>
</source>


<source>
    <url>https://ai-latest-technology.blogspot.com/</url>
    <highlights>['This blog is about Latest technology, AI and Machine learning. AI, machine learning and deep learning are getting involved in our day-to-day life. AI is improving our business, safety and comfort from mobile to automobile. AI technology, Bigdata, machine learning, deep learning, data science and data mining are starting the new era of technology.']</highlights>
</source>


<source>
    <url>https://learnaionlineatoz.bl

In [8]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert research assistant. You use xml-formatted context to research people's questions."),
    ("human", """
Please answer the following query based on the provided context. Please cite your sources at the end of your response.:
     
Query: {query}
---
<context>
{context}
</context>
""")
])

llm = ChatOpenAI()

chain = RunnableParallel({
    "query": RunnablePassthrough(),
    "context": retrieval_chain,
}) | generation_prompt | llm

In [10]:
chain.invoke("what is latest breakthrough in AI")

AIMessage(content='The latest breakthrough in AI involves the increasing integration of AI technology into our everyday lives. This includes advancements in areas such as business, safety, and comfort, from mobile devices to automobiles. Big data is playing a significant role in speeding up the development process of AI, paving the way for more sophisticated AI applications that will eventually have a significant impact on our daily lives.\n\nSources:\n1. AI Latest Technology Blog: https://ai-latest-technology.blogspot.com/\n2. AI 2080 Blog: http://ai2080.blogspot.com/2018/02/next-generation-technology.html')